In [4]:
import sys; sys.executable

'/usr/bin/python3'

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import MinkowskiEngine as ME 
from classresponsegeneration.kitti_dataset import KITTI_Dataset
from classresponsegeneration.models.BasicCNN_2d import BasicCNN
from classresponsegeneration.models.SPCNN_Minkowski import Basic_SPCNN, UNet

/usr/local/lib/python3.8/dist-packages/MinkowskiEngine-0.5.3-py3.8-linux-x86_64.egg/MinkowskiEngine/__init__.py:221: UserWarning: The MinkowskiEngine was compiled with CPU_ONLY flag. If you want to compile with CUDA support, make sure `torch.cuda.is_available()` is True when you install MinkowskiEngine.
  warnings.warn(


In [3]:
def train_SPCNN(device):
    params = {'batch_size': 64,
              'shuffle': True}

    train_set = KITTI_Dataset( mode='training', use_val=True )
    training_generator = torch.utils.data.DataLoader(train_set, **params)

    test_set = KITTI_Dataset(mode='testing', use_val=True)
    test_generator = torch.utils.data.DataLoader(test_set, **params)

    #model = Basic_SPCNN( in_feat=3, out_feat=1, D=2)
    model = UNet( in_feat=1, out_feat=1, D=2)
    #print(model)

    optimizer = torch.optim.SGD(model.parameters(), lr=0.2)
    loss_func = torch.nn.MSELoss()

    for epoch in range(5):
        print('epoch :',epoch)
        running_loss = 0.0

        for epoch in range(5):
        running_loss = 0.0
        print("epoch :", epoch)
        for i, data in enumerate(train_set):
            
            pc = data['pc']  # n x 3 (x,y,rel)
            crm_pc = data['crm_pc'] # n x 1
            
            # clear the gradient
            optimizer.zero_grad()
            
            input = ME.SparseTensor(pc[:,0:2], coordinates=pc[:,0:2].int())
            crm_input = ME.SparseTensor(crm_pc, coordinates=pc[:,0:2].int())
            
            #feed the input and acquire the output from network
            outputs = model(input)
            """"
            print("out:",outputs.shape)
            print("pc:",pc[:,0:2].int().shape)
            print("crm:",crm_input.features.shape)
            """
            #calculating the predicted and the expected loss
           
            loss = loss_func(outputs.features, crm_input.features)

            #compute the gradient
            loss.backward()

            #update the parameters
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 100 == 0:
                print('epoch: %d, data: %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss ))
                running_loss = 0.0


In [ ]:
if __name__ == '__main__':
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    #train_basicCNN()
    train_SPCNN(device)